# Обучаем персептрон распознавать классы вин

In [48]:
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.model_selection
import random

In [8]:
X,Y = sklearn.datasets.load_wine(return_X_y=True)
print(X.shape, Y.shape)

(178, 13) (178,)


In [40]:
Xsc = sklearn.preprocessing.StandardScaler().fit_transform(X)
Xsc1 = np.hstack([Xsc,np.ones_like(Xsc[:,0:1])])
Xsc1.shape


(178, 14)

In [41]:
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(Xsc,Y,stratify=Y, test_size=15)

In [42]:
def train(positive_examples, negative_examples, num_iterations = 100, eta = 1, report_frequency = 10):
    
    num_dims = positive_examples.shape[1]
    
    # Для простоты инициализируем веса нулями
    weights = np.zeros((num_dims,1)) 
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]
    
    for i in range(num_iterations):

        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)

        z = np.dot(pos, weights)   
        if z < 0: # позитивный пример классифицирован как негативный
            weights = weights + pos.reshape(weights.shape)

        z  = np.dot(neg, weights)
        if z >= 0: # негативный пример классифицирован как позитивный
            weights = weights - neg.reshape(weights.shape)
            
        # Periodically, print out the current accuracy on all examples 
        if i % report_frequency == 0:             
            pos_out = np.dot(positive_examples, weights)
            neg_out = np.dot(negative_examples, weights)        
            pos_correct = (pos_out >= 0).sum() / float(pos_count)
            neg_correct = (neg_out < 0).sum() / float(neg_count)
            print("Iteration={}, pos correct={}, neg correct={}".format(i,pos_correct,neg_correct))

    return weights

In [50]:
def select_samples(pos_label,neg_label,train=True):
    x,y = (train_x,train_y) if train else (test_x,test_y)
    pos = [x for x,y in zip(x,y) if y == pos_label]
    neg = [x for x,y in zip(x,y) if y == neg_label]
    return np.array(pos),np.array(neg)

pos,neg = select_samples(0,1)

wts = train(pos,neg)

Iteration=0, pos correct=0.46296296296296297, neg correct=0.6461538461538462
Iteration=10, pos correct=1.0, neg correct=0.9692307692307692
Iteration=20, pos correct=1.0, neg correct=0.9692307692307692
Iteration=30, pos correct=1.0, neg correct=0.9692307692307692
Iteration=40, pos correct=1.0, neg correct=0.9692307692307692
Iteration=50, pos correct=0.9444444444444444, neg correct=0.9846153846153847
Iteration=60, pos correct=0.9814814814814815, neg correct=0.9846153846153847
Iteration=70, pos correct=0.9814814814814815, neg correct=0.9846153846153847
Iteration=80, pos correct=0.9814814814814815, neg correct=0.9846153846153847
Iteration=90, pos correct=0.9814814814814815, neg correct=0.9846153846153847


In [61]:
pos_test, neg_test = select_samples(0,1,train=False)
correct = np.sum(pos_test@wts>0)+np.sum(neg_test@wts<0)
print(f"Accuracy={100*correct/(len(pos_test)+len(neg_test))}%")

Accuracy=100.0%
